[Link to the destination](#the_destination)

## NOW FOCUSED ON OWN-OCCUPIED SINGLE FAMILY

### updates since last version:

Added new features pertaining to aggregate stats. Best rfc for 2010-15 had nearly 50/50 accuracy

Now focus on mappping / visualization and timing effects

In [1]:
import pandas as pd
import numpy as np
import shapely
import fiona
from shapely import geometry
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import Point
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
import mplleaflet as mpl
import pysal as ps


In [2]:
# IMPORT MODULES FOR REGRESSION / CLASSIFICATION


from sklearn.preprocessing import PolynomialFeatures, StandardScaler, Imputer
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.cross_validation import cross_val_score, KFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.grid_search import GridSearchCV

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.svm import SVC



/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Start with most recent backup copy of the combined dataframe

In [3]:
model_file = pd.read_csv('/Users/DavidStier/Downloads/Model_File_CT-backup4.csv')

/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (171,172,212,224,231,256,306,309) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
model_file.shape

(92052, 365)

### Clean up 'legacy issues' from imported csv

In [5]:
first_pass_model_file_col_list = model_file.columns.tolist()
first_pass_model_file_dtypes_list = model_file.dtypes.tolist()
first_pass_model_file_first_Row = model_file[1:2]
first_pass_model_file_first_Row.to_csv('/Users/DavidStier/Downloads/first_pass_model_filerow1.csv')

In [6]:
## 
# Model file had included 2 features that were proxies for having installed solar
##

model_file.drop(['Valuation_all_sol_prmts'], inplace = True, axis = 1)
model_file.drop(['NUM_PERMITS_for_solar'], inplace = True, axis = 1)

## 
#  Model file 'neighborhood' and Outbuilding 1-3 columns had mixed content types - specifically the # zero was entered for some rows
##

model_file['neighborhood'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_1'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_2'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_3'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_3'].replace('0','Not Provided',inplace=True)

##
# Some census tract columns had 'nan' instead of NaN and Dtype needs to be Float
##

model_file['ctbdp040var_HC01_VC66'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC66'] = model_file['ctbdp040var_HC01_VC66'].astype(float)

model_file['ctbdp040var_HC01_VC67'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC67'] = model_file['ctbdp040var_HC01_VC67'].astype(float)

model_file['ctbdp040var_HC01_VC143'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC143'].replace(['4,000+'],4000, inplace = True) 
model_file['ctbdp040var_HC01_VC143'] = model_file['ctbdp040var_HC01_VC143'].astype(float)

model_file['ctbdp040var_HC01_VC151'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC151'].replace(['1,000+'],1000, inplace = True) 
model_file['ctbdp040var_HC01_VC151'].replace(['100-'],100, inplace = True) 
model_file['ctbdp040var_HC01_VC151'] = model_file['ctbdp040var_HC01_VC151'].astype(float)

model_file['ctbdp040var_HC01_VC185'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC185'].replace(['100-'],100, inplace = True) 
model_file['ctbdp040var_HC01_VC185'].replace(['2,000+'],2000, inplace = True) 
model_file['ctbdp040var_HC01_VC185'] = model_file['ctbdp040var_HC01_VC185'].astype(float)

model_file['ctb19049var_HD01_VD03'].replace(['2,500-'],2500, inplace = True) 
model_file['ctb19049var_HD01_VD03'].replace(['-'],0, inplace = True) 
model_file['ctb19049var_HD01_VD03'] = model_file['ctb19049var_HD01_VD03'].astype(float)

model_file['ctb19049var_HD01_VD06'].replace(['-'],0, inplace = True) 
model_file['ctb19049var_HD01_VD06'] = model_file['ctb19049var_HD01_VD06'].astype(float)

model_file['ctbdp040var_HC01_VC125'].replace(['1,000,000+'],1000000, inplace=True)  #nan and '-' and 1,000,000+
model_file['ctbdp040var_HC01_VC125'].replace(['-'],0, inplace = True)  #nan and '-' and 1,000,000+
model_file['ctbdp040var_HC01_VC125'] = model_file['ctbdp040var_HC01_VC125'].astype(float)



In [7]:
print model_file['NOT_Top2015PermTypdollars'].sum()
print model_file['NOT_Top2014PermTypdollars'].sum()
print model_file['NOT_Top2013PermTypdollars'].sum()
print model_file['NOT_Top2015PermTypNum'].sum()
print model_file['NOT_Top2014PermTypNum'].sum()
print model_file['NOT_Top2013PermTypNum'].sum()
print '>>>>>>>>>>'
print model_file['Top2015PermTypdollars'].sum()
print model_file['Top2014PermTypdollars'].sum()
print model_file['Top2013PermTypdollars'].sum()
print model_file['Top2015PermTypNum'].sum()
print model_file['Top2014PermTypNum'].sum()
print model_file['Top2013PermTypNum'].sum()


5488035078.6
3889717916.73
3376145884.57
28889.0
27021.0
24117.0
>>>>>>>>>>
1135287598.76
1057161193.9
955183399.51
8053.0
6262.0
6255.0


In [8]:
##
# Remove other permit activity if parcel installed solar that same year
# some homes may have had other electrical or structural work done at the same time as when they installed solar


reset_permit_info  =   {2016 : ['NOT_Top2016PermTypdollars','NOT_Top2016PermTypNum','Top2016PermTypdollars', 'Top2016PermTypNum'],
                        2015  : ['NOT_Top2015PermTypdollars','NOT_Top2015PermTypNum','Top2015PermTypdollars', 'Top2015PermTypNum'],
                        2014  : ['NOT_Top2014PermTypdollars','NOT_Top2014PermTypNum','Top2014PermTypdollars', 'Top2014PermTypNum'],
                        2013  : ['NOT_Top2013PermTypdollars','NOT_Top2013PermTypNum','Top2013PermTypdollars', 'Top2013PermTypNum'],
                        2012  : ['NOT_Top2012PermTypdollars','NOT_Top2012PermTypNum','Top2012PermTypdollars', 'Top2012PermTypNum'],
                        2011  : ['NOT_Top2011PermTypdollars','NOT_Top2011PermTypNum','Top2011PermTypdollars', 'Top2011PermTypNum'], 
                        2010  : ['NOT_Top2010PermTypdollars','NOT_Top2010PermTypNum','Top2010PermTypdollars', 'Top2010PermTypNum']}

for key, value in reset_permit_info.items():
    model_file.loc[model_file['Year_SOLAR'] == key,  value] = 0


In [9]:
##
# Check to make sure that the calc's affected the #s appropriately (eyeball check)
#

print model_file['NOT_Top2015PermTypdollars'].sum()
print model_file['NOT_Top2014PermTypdollars'].sum()
print model_file['NOT_Top2013PermTypdollars'].sum()
print model_file['NOT_Top2015PermTypNum'].sum()
print model_file['NOT_Top2014PermTypNum'].sum()
print model_file['NOT_Top2013PermTypNum'].sum()
print '>>>>>>>>>>'
print model_file['Top2015PermTypdollars'].sum()
print model_file['Top2014PermTypdollars'].sum()
print model_file['Top2013PermTypdollars'].sum()
print model_file['Top2015PermTypNum'].sum()
print model_file['Top2014PermTypNum'].sum()
print model_file['Top2013PermTypNum'].sum()


5451990761.01
3846780651.87
3345085742.17
27332.0
25924.0
23443.0
>>>>>>>>>>
1134857483.1
1052600682.56
947052223.86
7993.0
6216.0
6230.0


In [10]:
model_file['Year_SOLAR'].unique()

array([   nan,  2015.,  2011.,  2014.,  2016.,  2012.,  2013.,  2010.])

In [11]:
# model_file['ctbdp040var_HC01_VC125'].replace(['1,000,000+'],1000000, inplace=True)  #nan and '-' and 1,000,000+
# model_file['ctbdp040var_HC01_VC125'] = model_file['ctbdp040var_HC01_VC125'].astype(float)
# model_file['ctbdp040var_HC01_VC125']
# #model_file['ctbdp040var_HC01_VC125'].unique()
model_file['ctb19049var_HD01_VD06'].dtype

dtype('float64')

In [12]:
# save out a copy of the model datafile:
# model_file.to_csv('/Users/DavidStier/Downloads/Model_File-backup_TUESDAY.csv')
#model_file.to_json('/Users/DavidStier/Downloads/Model_Filebackup2.json')
# describe_file = model_file.describe()
# describe_file.to_csv('/Users/DavidStier/Downloads/Ctractcombined_described2_TUESDAY.csv')

## Bring in census tract and other shapefiles for leaflet

## Feature engineering - parcel level data

In [13]:
'''
# Possible new features to add based on all properties in addr_group:

model_file['resl_units_on_block'] = 
model_file['pct_block_resl_units_are_5plus_unit_bldgs']
model_file['pct_block_resl_units_are_3to4_unit_bldgs']
model_file['pct_block_resl_units_are_duplex']
model_file['pct_block_resl_units_are_SFR']
model_file['pct_block_SFR_OWN_OCC_units']
model_file['num_block_SFR_remodeled_pre_1980']
model_file['num_block_SFR_remodeled_1980_89']
model_file['num_block_SFR_remodeled_1990_99']
model_file['num_block_SFR_remodeled_2000_05']
model_file['num_block_SFR_remodeled_2006_10']
model_file['num_block_SFR_remodeled_2011_15']

ASSESSED VALUE percentiles
model_file['bldg_vs_land_value_ratio']
'''


"\n# Possible new features to add based on all properties in addr_group:\n\nmodel_file['resl_units_on_block'] = \nmodel_file['pct_block_resl_units_are_5plus_unit_bldgs']\nmodel_file['pct_block_resl_units_are_3to4_unit_bldgs']\nmodel_file['pct_block_resl_units_are_duplex']\nmodel_file['pct_block_resl_units_are_SFR']\nmodel_file['pct_block_SFR_OWN_OCC_units']\nmodel_file['num_block_SFR_remodeled_pre_1980']\nmodel_file['num_block_SFR_remodeled_1980_89']\nmodel_file['num_block_SFR_remodeled_1990_99']\nmodel_file['num_block_SFR_remodeled_2000_05']\nmodel_file['num_block_SFR_remodeled_2006_10']\nmodel_file['num_block_SFR_remodeled_2011_15']\n\nASSESSED VALUE percentiles\nmodel_file['bldg_vs_land_value_ratio']\n"

In [14]:
#model_file['tot_rooms_per_k_lot_sf'] = [model_file['R_TOTAL_RM'] / model_file['LAND_SF'] if x > 0 else 0 for x in model_file['LAND_SF']]


In [15]:
#model_file['bldg_vs_land_value_ratio'] = model_file['AV_BLDG'] / model_file['AV_LAND'] - did not use as it can go to infinity
model_file['home_age_in_2016'] = 2016 - model_file['YR_BUILT']
model_file['home_age_when_remodeled'] = model_file['YR_REMOD'] - model_file['YR_BUILT']
model_file['yrs_since_2016_when_remodeled'] = 2016 - model_file['YR_REMOD']
model_file['bathrooms_all_qty'] = model_file['R_FULL_BTH'] + model_file['R_HALF_BTH']
model_file['shape_vs_length'] = model_file['SHAPESTAre'] / model_file['SHAPESTLen']
model_file['tot_rooms_per_k_bldg_sf'] = model_file['R_TOTAL_RM'] / model_file['LIVING_ARE'] 
model_file['is_solar'] = [1 if x > 0 else 0 for x in model_file['Year_SOLAR'] ]

# Convert Owner Occupied to 0/1 from y/n so that it can be aggregated:
model_file['OWN_OCC'] = [1 if x == 'Y' else 0 for x in model_file['OWN_OCC'] ]

# Dummify certain categorical columns now:



#model_file['bldg_vs_land_size_ratio'] = model_file['LIVING_ARE'] / model_file['LAND_SF'] - this goes to infinity
#model_file['home_age_last_top_permitType_done'] = 
model_file.shape

(92052, 370)

In [16]:
Heat_Type_dummied = pd.get_dummies(model_file['Heat_Type'],prefix = 'Heat_Type_')
LU_dummied =pd.get_dummies(model_file['LU'],prefix = 'LU_')
R_ROOF_TYP_dummied = pd.get_dummies(model_file['R_ROOF_TYP'],prefix = 'R_ROOF_TYPE_')

type(Heat_Type_dummied)

pandas.core.frame.DataFrame

In [17]:
model_file = pd.concat([model_file, Heat_Type_dummied, LU_dummied, R_ROOF_TYP_dummied], axis = 1)

In [18]:
model_file.shape

(92052, 404)

### Aggregrate at addr_group, 1/2mi^2, census_tract and zip. 
#### Get count, mean, median, mode, sdev and sum for each level

### Select 'top' features to use for merging the aggregated calcs.  This will limit the # of new features
Based on early RandomForest Classifier results, have selected those features that had at least .002 contribution

In [19]:
top_aggr_features = ['addr_group','hmile_grid_cell','neighborhood','FIPS','NOT_Top2015PermTypNum','NOT_Top2015PermTypdollars','NOT_Top2014PermTypdollars',
                     'NOT_Top2013PermTypdollars','NOT_Top2014PermTypNum','GROSS_AREA','NOT_Top2013PermTypNum',
                     'SHAPESTLen','NOT_Top2012PermTypdollars','AV_LAND','home_age_when_remodeled','LIVING_ARE',
                     'home_age_in_2016','bldg_vs_land_value_ratio','tot_rooms_per_k_bldg_sf','LAND_SF','YR_REMOD',
                     'NOT_Top2012PermTypNum','YR_BUILT','shape_vs_length','OWN_OCC','hmile_grid_cell_R',
                     'yrs_since_2016_when_remodeled','Outbldg_1_Sqft','Exterior_Wall_Alum/Vinyl',
                     'NOT_Top2011PermTypdollars','R_TOTAL_RM','AV_BLDG','ctb1001var_HD01_VD41','ctb25093var_HD01_VD21',
                     'R_BDRMS','SHAPESTAre','st_num_group','Thr12014to2015_Street-Sidewalk','hmile_grid_cell_C',
                     'NOT_Top2010PermTypdollars','Thr12014to2015_excluded','R_FULL_BTH','PID','Heat_Type__Electric',
                     'LU__Residential Condo Unit','LU__Mixed Use Res/Comm', 'LU__Commercial','LU__Apartments 7 Units above','LU__Apartments 4-6 Units', 
                     'LU__Residential Single Family', 'LU__Residential Three Family', 'LU__Residential Two Family',
                     'LU__Residential Land', 'LU__Exempt','LU__Commercial Land', 'LU__Commercial Condo Unit','LU__Industrial',
                     'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Trees','Thr12014to2015_Trees','Thr12011to2013_Housing','Thr12011to2013_Building','Thr12014to2015_Building',
                     'Thr12014to2015_Housing','Thr12014to2015_Recycling-Pickup', 'is_solar','Thr12011_Recycling-Pickup','Thr12012_Recycling-Pickup','Thr12013_Recycling-Pickup','Thr12014_Recycling-Pickup','Thr12015_Recycling-Pickup', 'Thr12012_Trees','Thr12013_Trees' ,'Thr12014_Trees']

In [20]:
model_file_aggr_cols = pd.DataFrame()
for i in model_file.columns:
    if i in top_aggr_features:
        model_file_aggr_cols[i] = model_file[i]

In [21]:
model_file_aggr_cols.head(4).T

,0,1,2,3
AV_BLDG,235300,281200,244700,227800
AV_LAND,129900,121000,121400,121900
GROSS_AREA,3353,3299,3392,3108
LAND_SF,1150,1150,1150,1150
LIVING_ARE,2202,2307,2268,2028
OWN_OCC,1,0,0,0
PID,100001000,100002000,100003000,100004000
R_BDRMS,6,3,5,5
R_FULL_BTH,3,3,3,3
R_TOTAL_RM,12,9,13,11


In [22]:

model_file_addr_count = model_file_aggr_cols.groupby('addr_group',as_index = False).count()
#model_file_addr_num_recs = model_file.groupby('addr_group',as_index = False).size() # this is just the same value across all col's
model_file_addr_mean = model_file_aggr_cols.groupby('addr_group',as_index = False).mean()
model_file_addr_median = model_file_aggr_cols.groupby('addr_group',as_index = False).median()
#model_file_addr_mode = model_aggr_on.groupby('addr_group').mode()
#model_file_addr_sdev = model_aggr_on.groupby('addr_group',as_index = False).std()
model_file_addr_sum = model_file_aggr_cols.groupby('addr_group',as_index = False).sum()

model_file_halfMile_count = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).count()
model_file_halfMile_mean = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).mean()
model_file_halfMile_median = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).median()
#model_file_halfMile_mode = model_aggr_on.groupby('hmile_grid_cell',as_index = False).mode()
#model_file_halfMile_sdev = model_aggr_on.groupby('hmile_grid_cell',as_index = False).std()
model_file_halfMile_sum = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).sum()

model_file_nbrhood_count = model_file_aggr_cols.groupby('neighborhood',as_index = False).count()
model_file_nbrhood_mean = model_file_aggr_cols.groupby('neighborhood',as_index = False).mean()
model_file_nbrhood_median = model_file_aggr_cols.groupby('neighborhood',as_index = False).median()
#model_file_nbrhood_mode = model_aggr_on.groupby('neighborhood',as_index = False).mode()
#model_file_nbrhood_sdev = model_aggr_on.groupby('neighborhood',as_index = False).std()
model_file_nbrhood_sum = model_file_aggr_cols.groupby('neighborhood',as_index = False).sum()

model_file_centrct_count = model_file_aggr_cols.groupby('FIPS',as_index = False).count()
model_file_centrct_mean = model_file_aggr_cols.groupby('FIPS',as_index = False).mean()
model_file_centrct_median = model_file_aggr_cols.groupby('FIPS',as_index = False).median()
#model_file_centrct_mode = model_aggr_on.groupby('FIPS',as_index = False).mode()
#model_file_centrct_sdev = model_aggr_on.groupby('FIPS',as_index = False).std()
model_file_centrct_sum = model_file_aggr_cols.groupby('FIPS',as_index = False).sum()


"""
###

The sdev won't work if there are any string values in the df
##
"""

"\n###\n\nThe sdev won't work if there are any string values in the df\n##\n"

In [23]:
model_file.shape

(92052, 404)

### Create joined file aggregate level stats (eg 1 table for sum, count, median @ level = ____ and merge that to the main model file

In [24]:
# first, rename columns so that merge will not try to add suffixes (or not find) a column:
# Note, the 2 lists are for reference. I just cut and pasted into the lambda functions below:

aggregate_tables = ['model_file_addr_count','model_file_addr_mean','model_file_addr_median',
                    'model_file_halfMile_count','model_file_halfMile_mean','model_file_halfMile_median',
                    'model_file_nbrhood_count','model_file_nbrhood_mean','model_file_nbrhood_median',
                   'model_file_centrct_count', 'model_file_centrct_mean', 'model_file_centrct_median']

aggregate_suffixes = ['addr_ct','addr_av','add_md','hmil_ct','hmil_av','hmil_md',
                    'nbhd_ct','nbhd_av','nbhd_md','ctrt_ct', 'ctrt_av', 'ctrt_md']

# The lambda functions:
model_file_addr_count.rename(columns = lambda x: x + '_FOR_addr_cnt', inplace = True)
model_file_addr_mean.rename(columns = lambda x: x + '_FOR_addr_avg', inplace = True)
model_file_addr_median.rename(columns = lambda x: x + '_FOR_addr_med', inplace = True)
model_file_halfMile_count.rename(columns = lambda x: x + '_FOR_hmil_cnt', inplace = True)
model_file_halfMile_mean.rename(columns = lambda x: x + '_FOR_hmil_avg', inplace = True)
model_file_halfMile_median.rename(columns = lambda x: x + '_FOR_hmil_med', inplace = True)
model_file_nbrhood_count.rename(columns = lambda x: x + '_FOR_nbhd_cnt', inplace = True)
model_file_nbrhood_mean.rename(columns = lambda x: x + '_FOR_nbhd_avg', inplace = True)
model_file_nbrhood_median.rename(columns = lambda x: x + '_FOR_nbhd_med', inplace = True)
model_file_centrct_count.rename(columns = lambda x: x + '_FOR_ctrt_cnt', inplace = True)
model_file_centrct_mean.rename(columns = lambda x: x + '_FOR_ctrt_avg', inplace = True)
model_file_centrct_median.rename(columns = lambda x: x + '_FOR_ctrt_med', inplace = True)


In [25]:

# second, join one and two files as :
addr_group_step2 = pd.merge(model_file_addr_count, model_file_addr_median, left_on = 'addr_group_FOR_addr_cnt', right_on = 'addr_group_FOR_addr_med', how = 'left', suffixes=('', '_y'))
# # # add in third for :
addr_group_final = pd.merge(addr_group_step2, model_file_addr_mean, left_on = 'addr_group_FOR_addr_cnt', right_on ='addr_group_FOR_addr_avg', how = 'left' )
# # # merge the final to model_file
model_file_agg_ip = pd.merge(model_file, addr_group_final, left_on = 'addr_group', right_on ='addr_group_FOR_addr_cnt', how = 'left')

#repeat procedure for grids and merge to main file
grid_step2 = pd.merge(model_file_halfMile_count, model_file_halfMile_median,  left_on = 'hmile_grid_cell_FOR_hmil_cnt', right_on = 'hmile_grid_cell_FOR_hmil_med', how = 'left')
grid_final = pd.merge(grid_step2, model_file_halfMile_mean,  left_on = 'hmile_grid_cell_FOR_hmil_cnt', right_on = 'hmile_grid_cell_FOR_hmil_avg', how = 'left')
model_file_agg_ip2 = pd.merge(model_file_agg_ip, grid_final, left_on = 'hmile_grid_cell', right_on = 'hmile_grid_cell_FOR_hmil_cnt', how = 'left')



nhood_step2 = pd.merge(model_file_nbrhood_count, model_file_nbrhood_median, left_on = 'neighborhood_FOR_nbhd_cnt', right_on = 'neighborhood_FOR_nbhd_med', how = 'left')
nhood_final = pd.merge(nhood_step2, model_file_nbrhood_mean , left_on = 'neighborhood_FOR_nbhd_cnt', right_on = 'neighborhood_FOR_nbhd_avg', how = 'left')
model_file_agg_ip3 = pd.merge(model_file_agg_ip2, nhood_final, left_on = 'neighborhood', right_on = 'neighborhood_FOR_nbhd_cnt', how = 'left')

ctrt_step2 = pd.merge(model_file_centrct_count, model_file_centrct_median, left_on = 'FIPS_FOR_ctrt_cnt', right_on = 'FIPS_FOR_ctrt_med', how = 'left')
ctrt_final = pd.merge(ctrt_step2, model_file_centrct_mean, left_on = 'FIPS_FOR_ctrt_cnt', right_on = 'FIPS_FOR_ctrt_avg', how = 'left')
model_agg_ip4 = pd.merge(model_file_agg_ip3, ctrt_final, left_on = 'FIPS', right_on = 'FIPS_FOR_ctrt_cnt', how = 'left')

### delete 'in-process' files and rename final file back to model_file

In [26]:
del addr_group_step2
del addr_group_final
del model_file_agg_ip
del grid_step2
del grid_final
del model_file_agg_ip2
del nhood_step2
del nhood_final
del model_file_agg_ip3
del ctrt_step2
del ctrt_final


model_file = model_agg_ip4

In [27]:
model_file.shape

(92052, 1250)

In [28]:
del model_agg_ip4

## confirm all joined columns are ready for get_dummies and remove non-informative col's

In [29]:
aggregated_model_file_columns_list = model_file.columns.tolist()
aggregated_model_file_dtypes_list = model_file.dtypes.tolist()
aggregated_model_file_first_Row = model_file[1:2]
aggregated_model_file_first_Row.to_csv('/Users/DavidStier/Downloads/aggregated_model_filerow1.csv')
aggregated_model_file_dtypes_list

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 d

In [30]:
model_value_cols = model_file.select_dtypes(include=['float64', 'int64','int','float']).columns.tolist()

In [31]:
#model_value_cols

### drop aggregate 'name' columns 

In [32]:
aggr_name_cols_to_drop =[
 'addr_group_FOR_addr_cnt',
 'addr_group_FOR_addr_med',
 'addr_group_FOR_addr_avg',
 'hmile_grid_cell_FOR_hmil_cnt',
 'hmile_grid_cell_FOR_hmil_med',
 'hmile_grid_cell_FOR_hmil_avg',
 'neighborhood_FOR_nbhd_cnt',
 'neighborhood_FOR_nbhd_med',
 'neighborhood_FOR_nbhd_avg',
 'FIPS_FOR_ctrt_cnt',
 'FIPS_FOR_ctrt_med',
 'FIPS_FOR_ctrt_avg',
]

model_file.drop(aggr_name_cols_to_drop, inplace = True, axis = 1)

##   Add features based on aggregated stats

In [33]:
##
# Address group level:
Resl_Tax_parcels_per_addr = model_file['LU__Residential Single Family_FOR_addr_cnt'] + model_file['LU__Residential Two Family_FOR_addr_cnt'] + model_file['LU__Residential Three Family_FOR_addr_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_addr_cnt'] + model_file['LU__Apartments 7 Units above_FOR_addr_cnt'] + model_file['LU__Residential Condo Unit_FOR_addr_cnt']
Comml_Tax_parcels_per_addr = model_file['LU__Commercial_FOR_addr_cnt'] + model_file['LU__Commercial Land_FOR_addr_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_addr_cnt'] + model_file['LU__Commercial Condo Unit_FOR_addr_cnt'] + model_file['LU__Industrial_FOR_addr_cnt']
OTHER_Tax_parcels_per_addr = model_file['LU__Residential Land_FOR_addr_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_addr_cnt']
ALL_Tax_parcels_per_addr = Resl_Tax_parcels_per_addr + Comml_Tax_parcels_per_addr + OTHER_Tax_parcels_per_addr

##
# Half-mile group level:
Resl_Tax_parcels_per_hmil = model_file['LU__Residential Single Family_FOR_hmil_cnt'] + model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_hmil_cnt']
Comml_Tax_parcels_per_hmil = model_file['LU__Commercial_FOR_hmil_cnt'] + model_file['LU__Commercial Land_FOR_hmil_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_hmil_cnt'] + model_file['LU__Commercial Condo Unit_FOR_hmil_cnt'] + model_file['LU__Industrial_FOR_hmil_cnt']
OTHER_Tax_parcels_per_hmil = model_file['LU__Residential Land_FOR_hmil_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_hmil_cnt'] 
ALL_Tax_parcels_per_hmil = Resl_Tax_parcels_per_hmil + Comml_Tax_parcels_per_hmil + OTHER_Tax_parcels_per_hmil

##
# Neighborhood group level:
Resl_Tax_parcels_per_nbhd = model_file['LU__Residential Single Family_FOR_nbhd_cnt'] + model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] + model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] + model_file['LU__Residential Condo Unit_FOR_nbhd_cnt']
Comml_Tax_parcels_per_nbhd = model_file['LU__Commercial_FOR_nbhd_cnt'] + model_file['LU__Commercial Land_FOR_nbhd_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_nbhd_cnt'] + model_file['LU__Commercial Condo Unit_FOR_nbhd_cnt'] + model_file['LU__Industrial_FOR_nbhd_cnt']
OTHER_Tax_parcels_per_nbhd = model_file['LU__Residential Land_FOR_nbhd_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_nbhd_cnt'] 
ALL_Tax_parcels_per_nbhd = Resl_Tax_parcels_per_nbhd + Comml_Tax_parcels_per_nbhd + OTHER_Tax_parcels_per_nbhd

##
# Census Tract level:
Resl_Tax_parcels_per_ctrt = model_file['LU__Residential Single Family_FOR_ctrt_cnt'] + model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] + model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] + model_file['LU__Residential Condo Unit_FOR_ctrt_cnt']
Comml_Tax_parcels_per_ctrt = model_file['LU__Commercial_FOR_ctrt_cnt'] + model_file['LU__Commercial Land_FOR_ctrt_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_ctrt_cnt'] + model_file['LU__Commercial Condo Unit_FOR_ctrt_cnt'] + model_file['LU__Industrial_FOR_ctrt_cnt']
OTHER_Tax_parcels_per_ctrt = model_file['LU__Residential Land_FOR_ctrt_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_ctrt_cnt'] 
ALL_Tax_parcels_per_ctrt = Resl_Tax_parcels_per_ctrt + Comml_Tax_parcels_per_ctrt + OTHER_Tax_parcels_per_ctrt

In [34]:
##
# Address group level:

LU_Single_Family_pct_of_rsl_addr = model_file['LU__Residential Single Family_FOR_addr_cnt'] / Resl_Tax_parcels_per_addr
LU_Three_Six_family_pct_of_rsl_addr = (model_file['LU__Residential Two Family_FOR_addr_cnt'] + model_file['LU__Residential Three Family_FOR_addr_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_addr_cnt'] ) / Resl_Tax_parcels_per_addr
LU_Two_Family_pct_of_rsl_addr = model_file['LU__Residential Two Family_FOR_addr_cnt'] / Resl_Tax_parcels_per_addr
LU_Three_Family_pct_of_rsl_addr = model_file['LU__Residential Three Family_FOR_addr_cnt'] / Resl_Tax_parcels_per_addr
LU__Apts_4to6_pct_of_rsl_addr = model_file['LU__Apartments 4-6 Units_FOR_addr_cnt'] / Resl_Tax_parcels_per_addr
LU__Apts_7plus_pct_of_rsl_addr = model_file['LU__Apartments 7 Units above_FOR_addr_cnt'] /Resl_Tax_parcels_per_addr
LU__Resl_Condo_Unit_pct_of_rsl_addr = model_file['LU__Residential Condo Unit_FOR_addr_cnt'] /Resl_Tax_parcels_per_addr
LU_ALL_Multi_housing_pct_of_addr = (model_file['LU__Residential Two Family_FOR_addr_cnt'] + model_file['LU__Residential Three Family_FOR_addr_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_addr_cnt'] + model_file['LU__Apartments 7 Units above_FOR_addr_cnt'] + model_file['LU__Residential Condo Unit_FOR_addr_cnt']) / Resl_Tax_parcels_per_addr
LU__Mixed_Use_ResComml_pct_of_addr = model_file['LU__Mixed Use Res/Comm_FOR_addr_cnt'] / ALL_Tax_parcels_per_addr
LU__Comml_pct_of_addr = Comml_Tax_parcels_per_addr / ALL_Tax_parcels_per_addr 

##
# Half-mile group level:
LU_Single_Family_pct_of_rsl_hmil = model_file['LU__Residential Single Family_FOR_hmil_cnt'] / Resl_Tax_parcels_per_hmil
LU_Three_Six_family_pct_of_rsl_hmil = (model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] ) / Resl_Tax_parcels_per_hmil
LU_Two_Family_pct_of_rsl_hmil = model_file['LU__Residential Two Family_FOR_hmil_cnt'] / Resl_Tax_parcels_per_hmil
LU_Three_Family_pct_of_rsl_hmil = model_file['LU__Residential Three Family_FOR_hmil_cnt'] / Resl_Tax_parcels_per_hmil
LU__Apts_4to6_pct_of_rsl_hmil = model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] / Resl_Tax_parcels_per_hmil
LU__Apts_7plus_pct_of_rsl_hmil = model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] /Resl_Tax_parcels_per_hmil
LU__Resl_Condo_Unit_pct_of_rsl_hmil = model_file['LU__Residential Condo Unit_FOR_hmil_cnt'] /Resl_Tax_parcels_per_hmil
LU_ALL_Multi_housing_pct_of_hmil = (model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_hmil_cnt']) / Resl_Tax_parcels_per_hmil
LU__Mixed_Use_ResComml_pct_of_hmil = model_file['LU__Mixed Use Res/Comm_FOR_hmil_cnt'] / ALL_Tax_parcels_per_hmil
LU__Comml_pct_of_hmil = Comml_Tax_parcels_per_hmil / ALL_Tax_parcels_per_hmil 

##
# Neighborhood group level:
LU_Single_Family_pct_of_rsl_nbhd = model_file['LU__Residential Single Family_FOR_nbhd_cnt'] / Resl_Tax_parcels_per_nbhd
LU_Three_Six_family_pct_of_rsl_nbhd = (model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] ) / Resl_Tax_parcels_per_nbhd
LU_Two_Family_pct_of_rsl_nbhd = model_file['LU__Residential Two Family_FOR_nbhd_cnt'] / Resl_Tax_parcels_per_nbhd
LU_Three_Family_pct_of_rsl_nbhd = model_file['LU__Residential Three Family_FOR_nbhd_cnt'] / Resl_Tax_parcels_per_nbhd
LU__Apts_4to6_pct_of_rsl_nbhd = model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] / Resl_Tax_parcels_per_nbhd
LU__Apts_7plus_pct_of_rsl_nbhd = model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] /Resl_Tax_parcels_per_nbhd
LU__Resl_Condo_Unit_pct_of_rsl_nbhd = model_file['LU__Residential Condo Unit_FOR_nbhd_cnt'] /Resl_Tax_parcels_per_nbhd
LU_ALL_Multi_housing_pct_of_nbhd = (model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] + model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] + model_file['LU__Residential Condo Unit_FOR_nbhd_cnt']) / Resl_Tax_parcels_per_nbhd
LU__Mixed_Use_ResComml_pct_of_nbhd = model_file['LU__Mixed Use Res/Comm_FOR_nbhd_cnt'] / ALL_Tax_parcels_per_nbhd
LU__Comml_pct_of_nbhd = Comml_Tax_parcels_per_nbhd / ALL_Tax_parcels_per_nbhd 

## 
# Census tract level:
LU_Single_Family_pct_of_rsl_ctrt = model_file['LU__Residential Single Family_FOR_ctrt_cnt'] / Resl_Tax_parcels_per_ctrt
LU_Three_Six_family_pct_of_rsl_ctrt = (model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] ) / Resl_Tax_parcels_per_ctrt
LU_Two_Family_pct_of_rsl_ctrt = model_file['LU__Residential Two Family_FOR_ctrt_cnt'] / Resl_Tax_parcels_per_ctrt
LU_Three_Family_pct_of_rsl_ctrt = model_file['LU__Residential Three Family_FOR_ctrt_cnt'] / Resl_Tax_parcels_per_ctrt
LU__Apts_4to6_pct_of_rsl_ctrt = model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] / Resl_Tax_parcels_per_ctrt
LU__Apts_7plus_pct_of_rsl_ctrt = model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] /Resl_Tax_parcels_per_ctrt
LU__Resl_Condo_Unit_pct_of_rsl_ctrt = model_file['LU__Residential Condo Unit_FOR_ctrt_cnt'] /Resl_Tax_parcels_per_ctrt
LU_ALL_Multi_housing_pct_of_ctrt = (model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] + model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] + model_file['LU__Residential Condo Unit_FOR_ctrt_cnt']) / Resl_Tax_parcels_per_ctrt
LU__Mixed_Use_ResComml_pct_of_ctrt = model_file['LU__Mixed Use Res/Comm_FOR_ctrt_cnt'] / ALL_Tax_parcels_per_ctrt
LU__Comml_pct_of_ctrt = Comml_Tax_parcels_per_ctrt / ALL_Tax_parcels_per_ctrt 


In [35]:
## 
# Ratio of Shape vs. Len
##
model_file['shp_v_len_VS_addr_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_addr_avg']
model_file['shp_v_len_VS_nbhd_gr_AVG'] = model_file['shape_vs_length'] /  model_file['shape_vs_length_FOR_nbhd_avg']
model_file['shp_v_len_VS_hmil_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_hmil_avg']
model_file['shp_v_len_VS_ctrt_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_ctrt_avg']
model_file['shp_v_len_VS_addr_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_addr_med']
model_file['shp_v_len_VS_nbhd_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_nbhd_med']
model_file['shp_v_len_VS_hmil_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_hmil_med']
model_file['shp_v_len_VS_ctrt_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_ctrt_med']

##
#  Age of home in 2016
##
model_file['YR_BUILT_VS_addr_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_addr_avg']
model_file['YR_BUILT_VS_nbhd_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_nbhd_avg']
model_file['YR_BUILT_VS_hmil_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_hmil_avg']
model_file['YR_BUILT_VS_ctrt_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_ctrt_avg']
model_file['YR_BUILT_VS_addr_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_addr_med']
model_file['YR_BUILT_VS_nbhd_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_nbhd_med']
model_file['YR_BUILT_VS_hmil_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_hmil_med']
model_file['YR_BUILT_VS_ctrt_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_ctrt_med']

##
# GROSS AREA
#
model_file['GROSS_AREA_VS_addr_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_addr_avg']
model_file['GROSS_AREA_VS_nbhd_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_nbhd_avg']
model_file['GROSS_AREA_VS_hmil_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_hmil_avg']
model_file['GROSS_AREA_VS_ctrt_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_ctrt_avg']
model_file['GROSS_AREA_VS_addr_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_addr_med']
model_file['GROSS_AREA_VS_nbhd_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_nbhd_med']
model_file['GROSS_AREA_VS_hmil_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_hmil_med']
model_file['GROSS_AREA_VS_ctrt_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_ctrt_med']

##
# 311 recycling related calls
##

model_file['Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_addr_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_hmil_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_addr_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_addr_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_nbhd_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_hmil_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_hmil_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_ctrt_med']



## Create 'drop' lists for year-specific files

Basically, a year-specific file will not include any data that occurs in the future

In [36]:
YR_2016_DROPS =  ['Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
YR_2015_DROPS = [ 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
YR_2014_DROPS = ['Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
YR_2013_DROPS = ['Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
YR_2012_COL_DROPS = ['Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med','Top2012PermTypdollars', 'Top2012PermTypNum', 'Thr12012_Street-Sidewalk', 'Thr12012_Recycling-Pickup', 'Thr12012_excluded', 'Thr12012_Enviro', 'Thr12012_Trees', 'Thr12012_Building', 'Thr12012_Graffiti', 'Thr12012_Housing', 'NOT_Top2012PermTypdollars', 'NOT_Top2012PermTypNum', 'NOT_Top2012PermTypdollars_FOR_hmil_avg', 'NOT_Top2012PermTypNum_FOR_hmil_avg', 'NOT_Top2012PermTypdollars_FOR_addr_avg', 'NOT_Top2012PermTypNum_FOR_addr_avg', 'NOT_Top2012PermTypdollars_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_hmil_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_hmil_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_hmil_med', 'NOT_Top2012PermTypNum_FOR_hmil_med', 'NOT_Top2012PermTypdollars_FOR_nbhd_med', 'NOT_Top2012PermTypNum_FOR_nbhd_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_med', 'NOT_Top2012PermTypNum_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
YR_2011_COL_DROPS = ['Top2011PermTypdollars', 'Top2011PermTypNum', 'Thr12011_excluded', 'Thr12011_Recycling-Pickup', 'Thr12011to2013_Enviro', 'Thr12011_Trees', 'Thr12011_Enviro', 'Thr12011_Building', 'Thr12011_Street-Sidewalk', 'Thr12011_Graffiti', 'Thr12011_Housing', 'NOT_Top2011PermTypdollars', 'NOT_Top2011PermTypdollars_FOR_addr_avg', 'NOT_Top2011PermTypdollars_FOR_hmil_avg', 'NOT_Top2011PermTypdollars_FOR_ctrt_avg', 'NOT_Top2011PermTypdollars_FOR_nbhd_avg', 'NOT_Top2011PermTypNum', 'NOT_Top2011PermTypdollars_FOR_addr_cnt', 'NOT_Top2011PermTypdollars_FOR_hmil_cnt', 'NOT_Top2011PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2011PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2011PermTypdollars_FOR_addr_med', 'NOT_Top2011PermTypdollars_FOR_hmil_med', 'NOT_Top2011PermTypdollars_FOR_nbhd_med', 'NOT_Top2011PermTypdollars_FOR_ctrt_med', 'Top2012PermTypdollars', 'Top2012PermTypNum', 'Thr12012_Street-Sidewalk', 'Thr12012_Recycling-Pickup', 'Thr12012_excluded', 'Thr12012_Enviro', 'Thr12012_Trees', 'Thr12012_Building', 'Thr12012_Graffiti', 'Thr12012_Housing', 'NOT_Top2012PermTypdollars', 'NOT_Top2012PermTypNum', 'NOT_Top2012PermTypdollars_FOR_hmil_avg', 'NOT_Top2012PermTypNum_FOR_hmil_avg', 'NOT_Top2012PermTypdollars_FOR_addr_avg', 'NOT_Top2012PermTypNum_FOR_addr_avg', 'NOT_Top2012PermTypdollars_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_hmil_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_hmil_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_hmil_med', 'NOT_Top2012PermTypNum_FOR_hmil_med', 'NOT_Top2012PermTypdollars_FOR_nbhd_med', 'NOT_Top2012PermTypNum_FOR_nbhd_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_med', 'NOT_Top2012PermTypNum_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']


In [37]:
model_file.shape

(92052, 1270)

In [38]:
model_file['neighborhood'].value_counts()

Dorchester                                      16246
West Roxbury                                     9412
Hyde Park                                        8284
Roxbury                                          7836
Allston / Brighton                               6892
Greater Mattapan                                 6443
East Boston                                      6152
Jamaica Plain                                    5848
Roslindale                                       5833
South Boston / South Boston Waterfront           5114
Charlestown                                      2316
Downtown / Financial District                    1885
South End                                        1658
Back Bay                                         1418
Mission Hill                                     1256
Beacon Hill                                       956
Fenway / Kenmore / Audubon Circle / Longwood      951
Boston                                            261
South Boston                

In [39]:
print model_file.shape
low_neighborhoods = ['Allston','Beacon Hill','Boston','Brighton','Chestnut Hill','Downtown / Financial District','Fenway / Kenmore / Audubon Circle / Longwood','Mattapan','Mission Hill','South Boston']
model_file = model_file[~model_file['neighborhood'].isin(low_neighborhoods)]
print model_file.shape

(92052, 1270)
(86598, 1270)


In [40]:
##
# Save out 'Year Solar, PID and is_solar' to use for later analysis 
#model_file[['PID','Year_SOLAR','is_solar']].to_csv('/Users/DavidStier/Downloads/PID_YR_SOLAR_all.csv')
mf2016 = model_file[model_file['Year_SOLAR'] == 2016]
mf2016.to_csv('/Users/DavidStier/Downloads/YR_SOLARis2016.csv')

In [41]:
tf = model_file  # tf for test_file

In [42]:
tf.fillna(0, inplace = True)

### From here on, only SINGLE FAMILY OWNER OCCUPIED residential are included 

#### See prior notebooks for how to divide model_file for other home sizes


In [43]:
tf_r = tf[tf['resid_bldg']==True]
tf_r_single = tf_r[tf_r['LU'] == 'Residential Single Family']
print tf_r_single.shape, 'single family only'


(30033, 1270) single family only


In [44]:
tf_r_single = tf_r_single[tf_r_single['OWN_OCC'] == 1]
print tf_r_single.shape, 'single family only, now Owner Occ only'
# tfr_dtypes = tf_r.dtypes.tolist()
# tfrcolslist = tf_r.columns.tolist()


(25303, 1270) single family only, now Owner Occ only


<a id='the_destination'></a>

## The destination

## Segment into train / test / validate files

In [45]:
## 
#  Remove last remaining columns that should be excluded from the dummification
##

final_cols_to_drop = ['Unnamed: 0',
 'GIS_ID',
 'hmile_grid_cell',
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude',
 'YR_BUILT_VS_addr_gr_MED',  # these went to infinity, so will drop
'YR_BUILT_VS_hmil_gr_MED',
'YR_BUILT_VS_ctrt_gr_MED',
'GROSS_AREA_VS_addr_gr_MED',
'GROSS_AREA_VS_hmil_gr_MED',
'GROSS_AREA_VS_ctrt_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_addr_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_MED'
] 

tf_r_single.drop(final_cols_to_drop, inplace = True, axis = 1)


In [46]:
archive_cols = ['GIS_ID',
 'hmile_grid_cell',
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude' ]
##
# output a csv of the dropped columns for later audits
##
model_file[['GIS_ID',
 'hmile_grid_cell',
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude']].to_csv('/Users/DavidStier/Downloads/FinalColsDropped.csv')

KeyError: "['Qmile_grid_cell'] not in index"

In [ ]:
## 
# Create file specific column suffixes for the get_dummies functions
##

categ_cols_single = tf_r_single.select_dtypes(include=['object']).columns


In [ ]:
tf_r_single_dummied = pd.get_dummies(tf_r_single, columns = categ_cols_single)

print tf_r_single_dummied.shape, 'single family dummied'


## Run regression tests, holding out 2016 conversions

In [ ]:

validate_file_single = tf_r_single_dummied[tf_r_single_dummied['Year_SOLAR']==2016]
validate_file_single.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)


In [ ]:
train_test_combo_single = tf_r_single_dummied[tf_r_single_dummied['Year_SOLAR']!=2016]
print 'SINGLE FAMILY:  '
print validate_file_single.shape, ' validate file'
print train_test_combo_single.shape, ' all others'

In [ ]:
validate_file_single.to_csv('/Users/DavidStier/Downloads/Validate_full_fileWEDS11am.csv')


In [ ]:
Y_single = train_test_combo_single['is_solar']


### GET X

In [ ]:
tt_single_copy = train_test_combo_single.copy()
tt_single_copy.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)
X_single = tt_single_copy

print X_single.shape, ': X_single shape'

## Create 2016 train/test dataset

### Stratified Split Search

In [ ]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
# imp.fit(X)  - this was the original
# X2 = imp.transform(X)

imp.fit(X_single)
X2_single = imp.transform(X_single)

# Note - set X2 as variable for imputed "X" so that I could keep "X" clean

## Single Family Residential - Logit & RFC

In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.5, random_state=0)
sss.get_n_splits(X_single, Y_single)

for train_index, test_index in sss.split(X_single, Y_single):
    xtrain_single, xtest_single = X_single.iloc[train_index], X_single.iloc[test_index]
    ytrain_single, ytest_single = Y_single.iloc[train_index], Y_single.iloc[test_index]
    logit_single = LogisticRegression()
    logit_single.fit(xtrain_single, ytrain_single)

print xtrain_single.shape, ytrain_single.shape, xtest_single.shape, ytest_single.shape



In [ ]:
##
# send copy of feature labels to csv:
#
xtrain_single_column_labels = pd.DataFrame(xtrain_single[0:1])
#xtrain_single_column_labels
xtrain_single_column_labels.to_csv('/Users/DavidStier/Downloads/xtrain_single_column_labels_Weds12pm.csv')
xtrain_row_indexes_wAV_BLDG = pd.DataFrame(xtrain_single['AV_BLDG'])
xtrain_row_indexes_wAV_BLDG.to_csv('/Users/DavidStier/Downloads/xtrain_row_indexes_wAV_BLDG2pm.csv')

In [ ]:
logit_single.fit(X2_single, Y_single)
logit_single.get_params
logit_ypred_single = logit_single.predict(xtest_single)
cls_report_logit_single = classification_report(ytest_single, logit_ypred_single)
print cls_report_logit_single


In [ ]:
confusion_matrix(ytest_single,logit_ypred_single)


### Random Forest Classifier - Single family

rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4,criterion = 'gini',max_features = 100, random_state = 42)
precision    recall  f1-score   support

          0       0.97      1.00      0.99     14639
          1       0.84      0.25      0.38       539

avg / total       0.97      0.97      0.96     15178

[[14614    25]
 [  406   133]]
 
 roc_auc_score(ytest_single, rfc_ysingle_score[:, 0])
 0.030499703374509423

###
Run 2 (add min samples per leaf)
rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4,min_samples_leaf = 4,criterion = 'gini',max_features = 100, random_state = 42)
             precision    recall  f1-score   support

          0       0.97      1.00      0.98     14639
          1       0.83      0.19      0.31       539

avg / total       0.97      0.97      0.96     15178

[[14618    21]
 [  437   102]]
 
0.028939900671966678

### 
Run 3 (change to entropy)
             precision    recall  f1-score   support

          0       0.97      1.00      0.98     14639
          1       0.82      0.18      0.30       539

avg / total       0.97      0.97      0.96     15178

[[14617    22]
 [  440    99]]

###
rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4,min_samples_leaf = 4,criterion = 'entropy',max_features = 'sqrt', random_state = 42)

run 4 (max features to sqrt)
             precision    recall  f1-score   support

          0       0.97      1.00      0.98     14639
          1       1.00      0.02      0.04       539

avg / total       0.97      0.97      0.95     15178

[[14639     0]
 [  529    10]]
###
run 5 (max depth unlimited, gini, min samples_leaf:3 max feat .4
rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4,min_samples_leaf = 3,criterion = 'gini',max_features = .4, random_state = 42)

             precision    recall  f1-score   support

          0       0.98      1.00      0.99     14639
          1       0.81      0.52      0.63       539

avg / total       0.98      0.98      0.98     15178

[[14573    66]
 [  258   281]]
 0.018036490068147181
 
 
 AFTER DROPPING INTRA-YEAR PROJECT INFO:
 rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4,min_samples_leaf = 3,criterion = 'gini',max_features = .4, random_state = 42)
 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98     14353
          1       0.51      0.03      0.06       539

avg / total       0.95      0.96      0.95     14892

[[14336    17]
 [  521    18]]


In [ ]:
rfc_single = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=3,min_samples_leaf = 3,criterion = 'gini',max_features = .3, random_state = 42)


#scoresrfc = cross_val_score(rfc, X, Y, cv=10)
#scoresrfc(X)
rfc_single.fit(xtrain_single, ytrain_single)
rfc_single.get_params
rfc_ypred_single = rfc_single.predict(xtest_single)
cls_report_RFC_single = classification_report(ytest_single, rfc_ypred_single)
print cls_report_RFC_single

print confusion_matrix(ytest_single,rfc_ypred_single)



In [ ]:
rfc_ysingle_score = rfc_single.predict_proba(xtest_single)
roc_auc_score(ytest_single, rfc_ysingle_score[:, 1])

In [ ]:
rfc_ysingle_score[:, 1]

In [ ]:
rfc_single_features_df = pd.DataFrame(rfc_single.feature_importances_)
rfc_single_features_df.to_csv('/Users/DavidStier/Downloads/RFC_single_featuresWednesday2pm.csv')
rfc_single_predict_probabs = pd.DataFrame(rfc_ysingle_score)
rfc_single_predict_probabs.to_csv('/Users/DavidStier/Downloads/rfc_single_predict_probabs Wednesday2pm.csv')


In [ ]:
rfc_single_validate_predict_probs = pd.DataFrame(rfc_single.predict_proba(validate_file_single))
rfc_single_validate_predict_probs.to_csv('/Users/DavidStier/Downloads/rfc_single_VALIDATE_predict_probabs_WEDS2pm.csv')


In [ ]:
xtest_single.to_csv('/Users/DavidStier/Downloads/xtest_dfWEDNESDAY2pm.csv')
xtrain_single.to_csv('/Users/DavidStier/Downloads/xtrain_dfWEDNESDAY2pm.csv')
ytrain_single.to_csv('/Users/DavidStier/Downloads/ytrain_dfWEDNESDAY2pm.csv')
ytest_single.to_csv('/Users/DavidStier/Downloads/ytest_dfWEDNESDAY2pm.csv')

## Predict versus actual


In [ ]:
pause here

### GridSearch on Random Forest


In [ ]:
rfc_gs = RandomForestClassifier()

X, y = xtrain_single, ytrain_single
model = RandomForestClassifier(random_state=30)
param_grid = {"n_estimators"      : [100, 500, 1000, 5000],
              "criterion"         : ["gini", "entropy"],
              "max_features": ['sqrt'],
              "max_depth"         : [5, 10, 20],
              "min_samples_split" : [2, 4] ,
              "bootstrap": [True, False]}
grid_search = GridSearchCV(rfc_gs, param_grid, n_jobs=-1, cv=2)
grid_search.fit(X, y)

print grid_search.best_params_
#print confusion_matrix(ytest_SFR_meth1,rfc_ypred_SFR_meth1)

rfc_gs.get_params
rfc_gs_ypred_single = rfc_gs_single.predict(xtest_single)
cls_report_RFC_gs_single = classification_report(ytest_single, rfc_gs_ypred_single)
print cls_report_RFC_gs_single

print confusion_matrix(ytest_single,rfc_gs_ypred_single)

rfc_single_features_df = pd.DataFrame(rfc_gs.feature_importances_)
rfc_single_features_df.to_csv('/Users/DavidStier/Downloads/RFC_gs_single_featuresTUE.csv')




##  Drop all low-performing neighborhoods

In [ ]:
print tf_r_single.shape
SFR_meth1.shape


In [ ]:
SFR2 = SFR_meth1[~SFR_meth1['neighborhood'].isin(low_neighborhoods)]
SFR2.shape# for i in low_neighborhoods:
#     print i
#     SFR_target_1.drop(SFR_target_1['neighborhood']==[i], inplace = True, axis = 0)
# SFR_target_1.shape

In [ ]:
SFR_meth1 = SFR2
del SFR2

In [ ]:
SFR_meth1['neighborhood'].unique()

## Dummify and get splits

In [ ]:
categ_cols_SFR_meth1 = SFR_meth1.select_dtypes(include=['object']).columns
SFR_meth1_dummied = pd.get_dummies(SFR_meth1, columns = categ_cols_SFR_meth1)

print SFR_meth1_dummied.shape, 'sfr meth1 full file  dummied'
validate_file_SFR_meth1 = SFR_meth1_dummied[SFR_meth1_dummied['Year_SOLAR']==2016]
validate_file_SFR_meth1.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)

train_test_combo_SFR_meth1 = SFR_meth1_dummied[SFR_meth1_dummied['Year_SOLAR']!=2016]

## CREATE Y
Y_SFR_meth1 = train_test_combo_SFR_meth1['is_solar']

## CREATE X
tt_SFR_meth1_copy = train_test_combo_SFR_meth1.copy()
tt_SFR_meth1_copy.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)
X_SFR_meth1 = tt_SFR_meth1_copy

## CONFIRM SHAPES
print Y_SFR_meth1.shape, 'dummied Y for SFR target 1'
print X_SFR_meth1.shape, 'dummied shape of X for SFR target 1'

### perform sss (w/imputing)

In [ ]:
imp.fit(X_SFR_meth1)
X_SFR_meth1 = imp.transform(X_SFR_meth1)


In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.5, random_state=0)
sss.get_n_splits(X_SFR_meth1, Y_SFR_meth1)

for train_index, test_index in sss.split(X_SFR_meth1, Y_SFR_meth1):
    xtrain_SFR_meth1, xtest_SFR_meth1 = X_SFR_meth1[train_index], X_SFR_meth1[test_index]
    ytrain_SFR_meth1, ytest_SFR_meth1 = Y_SFR_meth1.iloc[train_index], Y_SFR_meth1.iloc[test_index]
    logit_SFR_meth1 = LogisticRegression()
    logit_SFR_meth1.fit(xtrain_SFR_meth1, ytrain_SFR_meth1)

print xtrain_SFR_meth1.shape, ytrain_SFR_meth1.shape, xtest_SFR_meth1.shape, ytest_SFR_meth1.shape


In [ ]:
logit_SFR_meth1.fit(X_SFR_meth1, Y_SFR_meth1)
logit_SFR_meth1.get_params
logit_ypred_SFR_meth1 = logit_SFR_meth1.predict(xtest_SFR_meth1)
cls_report_logit_SFR_meth1 = classification_report(ytest_SFR_meth1, logit_ypred_SFR_meth1)
print cls_report_logit_SFR_meth1


In [ ]:
confusion_matrix(ytest_SFR_meth1,logit_ypred_SFR_meth1)


### Random Forest Classifier - Single family Targeted neighborhoods

In [ ]:
rfc_SFR_meth1 = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_split=4)


#scoresrfc = cross_val_score(rfc, X, Y, cv=10)
#scoresrfc(X)
rfc_SFR_meth1.fit(xtrain_SFR_meth1, ytrain_SFR_meth1)
rfc_SFR_meth1.get_params
rfc_ypred_SFR_meth1 = rfc_SFR_meth1.predict(xtest_SFR_meth1)
cls_report_RFC_SFR_meth1 = classification_report(ytest_SFR_meth1, rfc_ypred_SFR_meth1)
print cls_report_RFC_SFR_meth1

print confusion_matrix(ytest_SFR_meth1,rfc_ypred_SFR_meth1)

rfc_SFR_meth1_features_df = pd.DataFrame(rfc_SFR_meth1.feature_importances_)
rfc_SFR_meth1_features_df.to_csv('/Users/DavidStier/Downloads/RFC_single_featuresTues.csv')


In [ ]:
X_SFR_meth1_COLS = pd.DataFrame(tt_SFR_meth1_copy.columns.tolist())
X_SFR_meth1_COLS.to_csv('/Users/DavidStier/Downloads/X_SFR_meth1_COLS_TUES.csv')

In [ ]:
rfc_SFR_meth1_y_score = rfc_SFR_meth1.predict_proba(xtest_SFR_meth1)
roc_auc_score(ytest_SFR_meth1, rfc_SFR_meth1_y_score[:, 0])

### GridSearch on Random Forest


In [ ]:
rfc = RandomForestClassifier()

X, y = xtrain_SFR_meth1, ytrain_SFR_meth1
model = RandomForestClassifier(random_state=30)
param_grid = {"n_estimators"      : [100, 500, 1000, 5000],
              "criterion"         : ["gini", "entropy"],
              "max_features": ['sqrt'],
              "max_depth"         : [5, 10, 20],
              "min_samples_split" : [2, 4] ,
              "bootstrap": [True, False]}
grid_search = GridSearchCV(rfc, param_grid, n_jobs=-1, cv=2)
grid_search.fit(X, y)

print grid_search.best_params_
#print confusion_matrix(ytest_SFR_meth1,rfc_ypred_SFR_meth1)





In [ ]:
bp1 = grid_search.best_params_
bp1['max_features'] = 'sqrt'

In [ ]:
rfc_SFR_meth1_best = RandomForestClassifier()

rfc_SFR_meth1_best.set_params(**bp1)

rfc_SFR_meth1_best.fit(X,y)    
rfc_ypred_SFR_meth1b = rfc_SFR_meth1_best.predict(xtest_SFR_meth1)

cls_report_RFC_SFR_meth1b = classification_report(ytest_SFR_meth1, rfc_ypred_SFR_meth1b)
print cls_report_RFC_SFR_meth1b

print confusion_matrix(ytest_SFR_meth1,rfc_ypred_SFR_meth1b)
rfc_SFR_meth1_best_features_df = pd.DataFrame(rfc_SFR_meth1_best.feature_importances_)

In [ ]:
single_predictions = rfc_SFR_meth1_best.predict_proba(xtest_SFR_meth1)

In [ ]:
single_predictions

In [ ]:
rfc_SFR_meth1_best_features_df
rfc_SFR_meth1_best_features_df.to_csv('/Users/DavidStier/Downloads/rfc_SFR_meth1_best_features_dfTUES.csv')

### Lasso

In [ ]:

lass = LassoCV(n_alphas = 500, cv=10)   # pass 500 random alphas
lass.fit(X,Y)

print pd.Series(lass.alphas_).describe()
print lass
print 'best alpha: ' , lass.alpha_


### Ridge

In [ ]:
ridg = linear_model.Ridge(alpha = .5)
ridg.fit(X, Y) 
print ridg.coef_
print  ridg.intercept_ 
print ridg.score(X, Y)

In [ ]:
# SFR_meth1_dummied.shape

In [ ]:
# SFR_meth1.to_csv('/Users/DavidStier/Downloads/SFR_meth1_fullFILE.csv')

In [ ]:
# xsingleMaxes = pd.DataFrame(X_single.max())
# xsingleMaxes.to_csv('/Users/DavidStier/Downloads/X_singlemaxex.csv')
X_single[0:4].to_csv('/Users/DavidStier/Downloads/X_singleColNames.csv')

In [ ]:
rfc_ysingle_score_df = pd.DataFrame(rfc_ysingle_score)
rfc_ysingle_score_df.to_csv('/Users/DavidStier/Downloads/rfc_ysingle_score_dfTues.csv')

In [ ]:
tf_r_single_dummied.shape

In [ ]:
bos_hmile_grids = pd.read_csv('/Users/DavidStier/Documents/Python and Data Science/DSI Class/Final Project/Model Building/YearSolar_XY_hmilegrids.csv')

In [ ]:
bos_hmile_grids.fillna(0, inplace = True)
bos_hmile_grids.info()

In [ ]:
bos_hmile_grids.drop(['Unnamed: 4','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'], inplace = True, axis = 1)

In [ ]:
bos_hmile_grids.dropna()
print bos_hmile_grids.shape

In [ ]:
# sb.pairplot(bos_hmile_grids, hue = 'WHEN')
# bos_hmile13_16=bos_hmile_grids[bos_hmile_grids['WHEN'] >2012]
# sb.pairplot(bos_hmile13_16, hue = "WHEN")
bos_hmile13_16.shape

In [ ]:
bos_hmile13_16.head(10)